In [1]:
import time
import re
from dateutil import parser
import yaml
import json
import hashlib

In [2]:
import pandas as pd
import os
import gzip

### LOAD CSV

In [4]:
def load_csv(path):
    return pd.read_csv(path, index_col='Unnamed: 0')

### LOAD YAML OR JSON FILE

In [3]:
def load_yaml(path):
    with open(path, mode='r') as file:
        return yaml.load(file, Loader=yaml.FullLoader)

In [4]:
def load_json(path):
    with open(path) as json_file:
        return json.load(json_file)

### HASH DATA WITH SHA256

In [3]:
def hash_data(data):
    
    # IF DATA IS A DICT, STRINGIFY IT
    if type(data) == dict:
        data = json.dumps(data, separators=(',', ':'))
    
    # ENCODE THE STRING WITH UTF8
    encoded = data.encode('utf-8')
    
    # HASH ENCODED DATA
    hashed = hashlib.sha256(encoded).hexdigest()
    
    return hashed

### CREATE DATASET

In [3]:
def create_dataset(main_dir):
    
    # CONTAINER
    container = []
    
    # SORTED SUB DIRECTORIES
    sub_dirs = os.listdir(main_dir)
    sub_dirs.sort()
    
    # LOOP THROUGH SUB-DIR
    for sub_dir in sub_dirs:
        
        # DIR PATH
        path = '{}/{}'.format(main_dir, sub_dir)
    
        # FIND & REVERSE SORT LOG FILES
        files = os.listdir(path)
        files.sort()
        files.reverse()
    
        # LOOP THROUGH LOG FILES
        for file in files:
            
            # FILE PATH
            path = '{}/{}/{}'.format(main_dir, sub_dir, file)

            # OPEN & READ THE COMPRESSED FILE
            with gzip.open(path, 'rb') as file:
                for line in file:

                    # DECODE AS STRING
                    decoded = line.decode('utf-8')

                    # PARSE LINE & EXTRACT PARAMS
                    module, code, message, timestamp = parse_line(decoded)

                    # APPEND TO CONTAINER
                    container.append([timestamp, module, code, message])
                    
    # CREATE DATAFRAME
    dataframe = pd.DataFrame(container, columns=[
        'timestamp',
        'module',
        'code',
        'message'
    ])
                
    return dataframe

### PARSE LINE INTO DICT

In [5]:
def parse_line(line):
    
    # FISH OUT THE CODE
    matches = re.findall(r"\[([0-9. ]*[0-9]+)\]", line)
    
    # DEFAULT TO NO CODE
    code = 'none'
    
    # IF A CODE MATCH IS FOUND, USE IT INSTEAD
    if len(matches) > 0:
        raw_code = matches[0]
        code = raw_code.replace(' ', '')
    
        # REMOVE THE BRACKETED CODE FROM THE LINE
        temp_block = '[{}]'.format(raw_code)
        line = line.replace(temp_block, '')
    
    # FIND ALL SPACES IN STRING
    spaces = [i for i, ltr in enumerate(line) if ltr == ' ']
    
    # FISH OUT THE DATE
    raw_date = line[:spaces[2]]
    timestamp = int(parser.parse(raw_date).timestamp())
    
    # FISH OUT THE MODULE
    raw_module = line[spaces[3]:spaces[4]]
    module = re.sub(r'[^A-Za-z]', '', raw_module)
    
    # FISH OUT THE REMAINING MESSAGE
    message = line[spaces[4]:].strip()
    message = re.sub(' +', ' ', message)
    
    return module, code, message, timestamp

### EXTRACT NUMBERS FROM MESSAGE

In [1]:
def extract_numbers(message):

    # EXTRACT NUMERIC VALUES
    values = re.findall(r"([0-9. ]*[0-9]+)", message)
    
    # SUBSTITUTE THEM WITH BRACKETS
    substituted = re.sub(r"([0-9. ]*[0-9]+)", '{}', message)
    
    return values, substituted

### SAVE DATA TO FILE

In [4]:
def save_data(data, path):
    with open(path, 'w') as file:
        file.write(data)

### FILTER ZEROES FROM LIST

In [1]:
def filter_backlog(data):
    
    # FILTER ZEROS
    filtered = filter(lambda x: x != '0x0000000000000000000000000000000000000000', data)
    
    # CONVERT TO LIST & RETURN
    return list(filtered)